In [1]:
from google.colab import files
files.upload()

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!pip install kaggle -q

In [ ]:
!kaggle datasets download -d jfcaro/5000-transcripts-of-youtube-ai-related-videos --unzip

Dataset URL: https://www.kaggle.com/datasets/jfcaro/5000-transcripts-of-youtube-ai-related-videos
License(s): unknown
  0% 0.00/36.0M [00:00<?, ?B/s]
100% 36.0M/36.0M [00:00<00:00, 828MB/s]


In [ ]:
!ls

kaggle.json  sample_data  YouTube_transcripts_Kaggle.csv


In [ ]:
import pandas as pd
df = pd.read_csv('YouTube_transcripts_Kaggle.csv')
print(df.head())

                                               title             author  \
0            Introduction to Vertex AI Feature Store  Google Cloud Tech   
1  Illuminating the global fishing fleet through ...  Google Cloud Tech   
2                 Hyperparameter Tuning on Vertex AI  Google Cloud Tech   
3                    End-to-end MLOps with Vertex AI  Google Cloud Tech   
4  Transformers, explained: Understand the model ...  Google Cloud Tech   

                                          transcript  \
0  PRIYANKA VERGADIA:\nDid you know that most of ...   
1  SPEAKER 1: Oceanic\necosystems are threatened ...   
2  PRIYANKA VERGADIA: Hi. I'm Priyanka Vergadia. ...   
3  PRIYANKA VERGADIA: One of the\nbiggest challen...   
4  [MUSIC PLAYING] DALE MARKOWITZ: The\nneat thin...   

                               playlist_name  
0  AI and Machine Learning with Google Cloud  
1  AI and Machine Learning with Google Cloud  
2  AI and Machine Learning with Google Cloud  
3  AI and Machine Learni

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cuda


In [ ]:
!pip install -q sentence-transformers

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.6 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
print(df.columns)

Index(['title', 'author', 'transcript', 'playlist_name'], dtype='object')


In [ ]:
def chunk_text(text, max_words=100):
    words = text.split()
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

texts = []
metadatas = []

for idx, row in df.iterrows():
    chunks = chunk_text(row['transcript'])
    for i, chunk in enumerate(chunks):
        texts.append(chunk)
        metadatas.append({
            'title': row.get('title', ''),
            'author': row.get('author', ''),
            'chunk_index': i,
            'playlist_name': row.get('playlist_name', '')
        })


In [ ]:
import torch

embeddings = model.encode(texts, convert_to_tensor=True, device=device)
print(f"Encoded {len(texts)} text chunks into embeddings of shape: {embeddings.shape}")


Encoded 232543 text chunks into embeddings of shape: torch.Size([232543, 384])


In [ ]:
def search(query, top_k=5):
    query_embedding = model.encode(query, convert_to_tensor=True, device=device)
    cos_scores = torch.nn.functional.cosine_similarity(query_embedding, embeddings)
    top_results = torch.topk(cos_scores, k=top_k)

    results = []
    for score, idx in zip(top_results.values.cpu().numpy(), top_results.indices.cpu().numpy()):
        meta = metadatas[idx]
        results.append((score, meta))
    return results


In [ ]:
# Example query
query = "Machine Learning"
results = search(query)

for score, meta in results:
    print(f"Score: {score:.4f}, Title: {meta['title']}, Author: {meta['author']}, Chunk: {meta['chunk_index']}")


Score: 0.7978, Title: Intro to Machine Learning Lesson 2: Basic Data Exploration | Kaggle, Author: Kaggle, Chunk: 12
Score: 0.6277, Title: Lecture 10.4 — Advice For Applying Machine Learning | Diagnosing Bias Vs Variance — [Andrew Ng], Author: Artificial Intelligence - All in One, Chunk: 14
Score: 0.6259, Title: Linear Systems of Equations, Author: Steve Brunton, Chunk: 16
Score: 0.6115, Title: Lecture 2.3 — A geometrical view of perceptrons  [Neural Networks for Machine Learning], Author: Colin Reckons, Chunk: 10
Score: 0.6059, Title: What is Machine Learning?, Author: Google Cloud Tech, Chunk: 1
